### Plugins for Google Cloud Platform (GCP)
- reference link: https://github.com/vmware-tanzu/velero-plugin-for-gcp#setup

### setup tasks
- Create an GCS bucket
- Set permissions for Velero
- Install and start Velero

### Create an GCS bucket
- BUCKET=<YOUR_BUCKET>
    gsutil mb gs://$BUCKET/
    - specify your bucket name and then use gsutil mb command line to make a bucket
    - gsutil (google storage utility) is command line tool which help you to manage google cloud storage object
    - mb stand for make bucket

### Set permissions for Velero
- get project id
    - gcloud config list
        - to list current config with command: gcloud config list
    - PROJECT_ID=$(gcloud config get-value project)
        - get project id with command: gcloud config get-value project
- create service account
    - GSA_NAME=velero
    gcloud iam service-accounts create $GSA_NAME \
        --display-name "Velero service account"
        - gcloud iam service-accounts create to create service accout
- list current service account
    - gcloud iam service-accounts list
- service account attribute
    - name
    - email
    - note: in this case, we use service account email as an id for a service account
- SERVICE_ACCOUNT_EMAIL=$(gcloud iam service-accounts list \
  --filter="displayName:Velero service account" \
  --format 'value(email)')
    - get service account with --filter and --format flag

### Create role, binding service account to role, binding service account to objectAdmin role of specific bucket object
- config role
    - ROLE_PERMISSIONS=(
    compute.disks.get
    compute.disks.create
    compute.disks.createSnapshot
    compute.projects.get
    compute.snapshots.get
    compute.snapshots.create
    compute.snapshots.useReadOnly
    compute.snapshots.delete
    compute.zones.get
    storage.objects.create
    storage.objects.delete
    storage.objects.get
    storage.objects.list
    iam.serviceAccounts.signBlob
)
- create role
    - gcloud iam roles create velero.server \
    --project $PROJECT_ID \
    --title "Velero Server" \
    --permissions "$(IFS=","; echo "${ROLE_PERMISSIONS[*]}")"
        - gcloud iam roles create to create role
- add iam policy binding at project level
    - gcloud project add-iam-policy-binding
- binding objectAdmin role of a given bucket to velero service account
    - gsutil iam ch serviceAccount:$SERVICE_ACCOUNT_EMAIL:objectAdmin gs://${BUCKET}

### Grant access and install Velero
- create service account key
    - gcloud iam service-accounts keys create credentials-velero \
    --iam-account $SERVICE_ACCOUNT_EMAIL
        - service account key is key to identify and authenticate program or machine which is assoicate to that service account
- velero install \
    --provider gcp \
    --plugins velero/velero-plugin-for-gcp:v1.6.0 \
    --bucket $BUCKET \
    --secret-file ./credentials-velero
    - --secret-file is flag to specify service-account key. in this case service-account key is a filename credential-velero
